# Goal

To Web Scrape Zillow to see the average home price of the Dallas Area. ***Note that API is limited to pulling 500 Values***

### Links

Zillow: https://www.zillow.com/dallas-tx/

Zillow Sold: https://www.zillow.com/dallas-tx/sold/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A33.041254272695596%2C%22south%22%3A32.594037410567665%2C%22east%22%3A-96.46741105322266%2C%22west%22%3A-97.08745194677735%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A38128%2C%22regionType%22%3A6%7D%5D%7D


### Imports

In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
import requests

pd.set_option("display.max_columns", None)

### Functions

In [1]:
def get_listings(api_key, listing_url):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"

    querystring = {
        "api_key": api_key,
        "url":listing_url
    }

    return requests.request("GET", url, params=querystring)

def get_property_detail(api_key, zpid):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/property"

    querystring = {
        "api_key": api_key,
        "zpid":zpid
    }

    return requests.request("GET", url, params=querystring)

def get_zpid(api_key, street, city, state, zip_code=None):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/zpidByAddress"

    querystring = {
        "api_key": api_key,
        "street": street,
        "city": city,
        "state": state,
        "zip_code":zip_code
    }

    return requests.request("GET", url, params=querystring)

def convert_sqft_to_acres(dfs):
    # Conversion factor from square feet to acres
    sqft_to_acres = 1 / 43560

    # Using the .loc accessor to modify rows where the Lot Area Unit is 'sqft'
    # Convert and round the area to three decimal places
    dfs.loc[dfs['Lot Area Unit'] == 'sqft', 'Lot Area'] = (dfs['Lot Area'] * sqft_to_acres).round(3)
    dfs.loc[dfs['Lot Area Unit'] == 'sqft', 'Lot Area Unit'] = 'acres'
    
    return dfs


### Locals and Constants

In [2]:
api_key = "220f3c4c-6532-416a-99b2-8b3e85893f8e"

### Data

In [3]:
sold_url = "https://www.zillow.com/dallas-tx/sold/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A33.041254272695596%2C%22south%22%3A32.594037410567665%2C%22east%22%3A-96.46741105322266%2C%22west%22%3A-97.08745194677735%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A38128%2C%22regionType%22%3A6%7D%5D%7D"

# get listings
sold_response = get_listings(api_key, sold_url)

NameError: name 'requests' is not defined

In [4]:
# view all keys
sold_response.json().keys()

# check if request is successful
sold_response.json()["is_success"]

# view count of properies returned in request
num_of_properties = sold_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

NameError: name 'sold_response' is not defined

In [ ]:
# view all listings
df_listings = pd.json_normalize(sold_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_listings))
print("Number of columns:", len(df_listings.columns))
df_listings

In [ ]:
# Export DataFrame to CSV file
df_listings.to_csv('listings_raw_sold.csv', index=False)

## Clean Up Data Frame

In [ ]:
df = pd.read_csv("listings_raw_sold.csv")

# Clean up column names
df.columns = df.columns.str.lower()  # Convert to lowercase
df.columns = df.columns.str.replace(' ', '_')  # Replace spaces with underscores
df.columns = df.columns.str.replace('[^a-zA-Z0-9_]', '')  # Remove special characters

# Display the list of column names
print("Column names:")
print(df.columns.tolist())

In [ ]:
# Keep specific columns
columns_to_keep = [
    'detailurl', 'statustext', 'area',

    'hdpdata.homeinfo.zpid', 'hdpdata.homeinfo.streetaddress', 'hdpdata.homeinfo.zipcode', 'hdpdata.homeinfo.city', 'hdpdata.homeinfo.state', 
    'hdpdata.homeinfo.latitude', 'hdpdata.homeinfo.longitude', 'hdpdata.homeinfo.price', 'hdpdata.homeinfo.datepricechanged', 
    'hdpdata.homeinfo.bathrooms', 'hdpdata.homeinfo.bedrooms', 'hdpdata.homeinfo.livingarea', 'hdpdata.homeinfo.hometype', 
    'hdpdata.homeinfo.homestatus', 'hdpdata.homeinfo.daysonzillow', 

    'hdpdata.homeinfo.zestimate', 'hdpdata.homeinfo.rentzestimate',

    'hdpdata.homeinfo.pricechange',

    'hdpdata.homeinfo.taxassessedvalue', 'hdpdata.homeinfo.lotareavalue', 'hdpdata.homeinfo.lotareaunit',

    # MIGHT BE MORE
    
    ]
df[columns_to_keep]
# Select columns to keep
df = df[columns_to_keep]
df

In [ ]:
column_name_mapping = {
    'detailurl' : 'URL', 
    'statustext' : 'Status', 
    'hdpdata.homeinfo.zpid' : 'Zip ID', 
    'area': "Sqft",
    'hdpdata.homeinfo.streetaddress' : 'Street Address', 
    'hdpdata.homeinfo.zipcode' : 'Zip Code', 
    'hdpdata.homeinfo.city' : 'City', 
    'hdpdata.homeinfo.state' : 'State', 
    'hdpdata.homeinfo.latitude' : 'Latitude', 
    'hdpdata.homeinfo.longitude' : 'Longitude', 
    'hdpdata.homeinfo.price' : 'Price', 
    'hdpdata.homeinfo.datepricechanged' : 'Date Price Changed', 
    'hdpdata.homeinfo.bathrooms' : 'Baths', 
    'hdpdata.homeinfo.bedrooms' : 'Beds', 
    'hdpdata.homeinfo.livingarea' : 'Living Area', 
    'hdpdata.homeinfo.hometype' : 'Home Type', 
    'hdpdata.homeinfo.homestatus' : 'Home Status', 
    'hdpdata.homeinfo.daysonzillow' : 'Days on Zillow', 

    'hdpdata.homeinfo.zestimate' : 'Zestimate', 
    'hdpdata.homeinfo.rentzestimate' : 'Rent Zestimate',

    'hdpdata.homeinfo.pricechange' : 'Price Change',

    'hdpdata.homeinfo.taxassessedvalue' : 'Tax Assessed Value', 
    'hdpdata.homeinfo.lotareavalue' : 'Lot Area', 
    'hdpdata.homeinfo.lotareaunit' : 'Lot Area Unit',


    # MIGHT BE MORE
    
}

# Rename columns
df.rename(columns=column_name_mapping, inplace=True)

In [ ]:
# Assuming dfs is your DataFrame
number_of_rows, number_of_columns = df.shape

print("Number of rows:", number_of_rows)
print("Number of columns:", number_of_columns)

df

In [ ]:
# Convert Lot Values from Sqft to Acres
# Call the function to convert areas
df = convert_sqft_to_acres(df)

In [ ]:
# Export DataFrame to CSV file
df.to_csv('listings_cleaned_sold.csv', index=False)